<a href="https://colab.research.google.com/github/Ishannaik/PDF-To-Audiobook-converter/blob/main/Book_to_audiobook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main Section


In [ ]:
!pip install pymupdf pydub requests
!pip install elevenlabs -U

In [ ]:
import requests

def get_eleven_labs_voices(api_key):
    url = "https://api.elevenlabs.io/v1/voices"
    headers = {
        "xi-api-key": api_key
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()["voices"]
    else:
        print(f"Error: {response.status_code}")
        return None

# Usage
api_key = "INSERT API KEY HERE"
voices = get_eleven_labs_voices(api_key)
if voices:
    for voice in voices:
        print(f"{voice['name']} - {voice['voice_id']}")
else:
    print("Could not retrieve voices.")


# RUN ME ->

In [ ]:
import os
import re
import shutil
import fitz  # PyMuPDF
import time
from google.colab import drive
from elevenlabs import generate, set_api_key
from pydub import AudioSegment

# Connect to Google Drive if not connected
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')

# Load the PDF text
pdf_path = 'INSERT PATH HERE'  # Ensure this path is correct
pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]  # Extract the file name (excluding extension)

# Create necessary folders in Colab for storing chunks and the complete audiobook
chunks_colab_folder = f'/content/AUDIOBOOK/Chunks/{pdf_file_name}/'
audiobook_colab_folder = f'/content/AUDIOBOOK/Audiobook/{pdf_file_name}/'

os.makedirs(chunks_colab_folder, exist_ok=True)
os.makedirs(audiobook_colab_folder, exist_ok=True)

# Create necessary folders in Google Drive for storing chunks and the complete audiobook
chunks_drive_folder = f'/content/drive/MyDrive/Audiobooks/Chunks/{pdf_file_name}/'
audiobook_drive_folder = f'/content/drive/MyDrive/Audiobooks/Audiobook/{pdf_file_name}/'

os.makedirs(chunks_drive_folder, exist_ok=True)
os.makedirs(audiobook_drive_folder, exist_ok=True)

# Set your API key (replace <YOUR_API_KEY> with your actual API key)
set_api_key("INSERT API KEY HERE")

def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        print(f"Text extracted from PDF successfully, length: {len(text)} characters")
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

def upload_missing_chunks_to_drive():
    colab_chunk_files = set(os.listdir(chunks_colab_folder))
    drive_chunk_files = set(os.listdir(chunks_drive_folder))
    missing_chunks = colab_chunk_files - drive_chunk_files

    if missing_chunks:
        print(f"Uploading missing chunks to Google Drive...")
        for chunk_filename in missing_chunks:
            chunk_colab_path = os.path.join(chunks_colab_folder, chunk_filename)
            chunk_drive_path = os.path.join(chunks_drive_folder, chunk_filename)
            shutil.copy(chunk_colab_path, chunk_drive_path)
        print(f"Uploaded {len(missing_chunks)} missing chunks to Google Drive.")

def download_missing_chunks_to_colab():
    colab_chunk_files = set(os.listdir(chunks_colab_folder))
    drive_chunk_files = set(os.listdir(chunks_drive_folder))
    missing_chunks = drive_chunk_files - colab_chunk_files

    if missing_chunks:
        print(f"Downloading missing chunks to Colab...")
        for chunk_filename in missing_chunks:
            chunk_drive_path = os.path.join(chunks_drive_folder, chunk_filename)
            chunk_colab_path = os.path.join(chunks_colab_folder, chunk_filename)
            shutil.copy(chunk_drive_path, chunk_colab_path)
        print(f"Downloaded {len(missing_chunks)} missing chunks to Colab.")

def split_text_into_chunks(text):
    chunks = []
    max_chunk_length = 5000  # Maximum length for a chunk
    current_chunk = ""

    for sentence in re.split(r'(?<=[.!?])\s+', text):
        if len(current_chunk) + len(sentence) <= max_chunk_length:
            current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def generate_mp3(text_chunks, voice_id, start=1):
    i = start
    total_duration = 0
    total_characters = len("".join(text_chunks))
    total_word_count = len(re.findall(r'\w+', "".join(text_chunks)))
    total_word_left = total_word_count

    for chunk in text_chunks:
        print(f"Generating MP3 for chunk {i}...")
        chunk_mp3_file_path = os.path.join(chunks_colab_folder, f'{i}.mp3')

        if os.path.exists(chunk_mp3_file_path):
            print(f"MP3 for chunk {i} already exists, skipping...")
            i += 1
            continue

        try:
            start_time = time.time()
            audio = generate(
                text=chunk,
                voice=voice_id,
                model='eleven_multilingual_v1'
            )
            end_time = time.time()
            duration_ms = len(audio)
            duration_sec = duration_ms / 1000
            word_count = len(re.findall(r'\w+', chunk))
            total_duration += duration_sec
            total_characters -= len(chunk)
            total_word_left -= word_count

            with open(chunk_mp3_file_path, 'wb') as mp3_file:
                mp3_file.write(audio)

            print(f"MP3 for chunk {i} generated successfully.")
            print(f"Duration: {duration_sec} seconds")
            print(f"Word Count: {word_count} words")
            print(f"Total Duration So Far: {total_duration} seconds")
            print(f"Characters Left: {total_characters} characters")
            print(f"Words Left: {total_word_left} words")

            save_to_drive(chunk_mp3_file_path, chunks_drive_folder)
            print(f"MP3 for chunk {i} uploaded to Google Drive (Chunks).")

        except Exception as e:
            print(f"Exception occurred while generating MP3 for chunk {i}: {e}")
            break
        i += 1

def save_to_drive(file_path, drive_folder):
    try:
        shutil.copy(file_path, drive_folder)
        print(f"File uploaded to Google Drive: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Error uploading to Google Drive: {e}")

# Load the PDF text
text = extract_text_from_pdf(pdf_path)

if text:
    # Split the text into chunks while preserving sentences
    text_chunks = split_text_into_chunks(text)

    # Upload missing chunks to Google Drive
    upload_missing_chunks_to_drive()

    # Download missing chunks to Colab
    download_missing_chunks_to_colab()

    # Generate MP3 files from text chunks
    generate_mp3(text_chunks, voice_id='INSERT VOICE ID HERE')

    # Calculate the total duration of the audiobook
    total_duration = 0
    for i in range(1, len(text_chunks) + 1):
        chunk_mp3_file_path = os.path.join(chunks_colab_folder, f'{i}.mp3')
        if os.path.exists(chunk_mp3_file_path):
            chunk_audio = AudioSegment.from_mp3(chunk_mp3_file_path)
            total_duration += len(chunk_audio) / 1000  # Convert milliseconds to seconds

    print("Audiobook generation complete.")
    print(f"Total Duration: {total_duration} seconds")
else:
    print("No text extracted from PDF; cannot proceed with audiobook generation.")


# COMBINE UNCOMBINED CHUNKS


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pydub import AudioSegment
import os

# Specify the path to the directory containing your MP3 files
audio_directory = 'INSERT PATH HERE'

# Get the folder name
folder_name = os.path.basename(audio_directory)

# Get a list of all MP3 files in the directory
audio_files = [f for f in os.listdir(audio_directory) if f.endswith('.mp3')]

# Sort the files in case they are not in order
audio_files.sort()

# Check if there are any files to combine
if len(audio_files) > 0:
    # Create an empty AudioSegment to store the combined audio
    combined_audio = AudioSegment.empty()

    # Initialize a progress message
    progress_message = "Combining:"

    # Load each audio file, add it to the combined audio, and update the progress message
    for i, file in enumerate(audio_files, start=1):
        audio_path = os.path.join(audio_directory, file)
        audio = AudioSegment.from_mp3(audio_path)
        combined_audio += audio
        progress_message += f" {i}"

    # Export the combined audio to a file with the folder name
    combined_filename = f'{folder_name}.mp3'
    combined_audio.export(os.path.join(audio_directory, combined_filename), format='mp3')

    print(f'Combined {len(audio_files)} audio files into "{combined_filename}"')
    print(progress_message)
else:
    print('No audio files to combine.')


# Split into Half

In [ ]:
from pydub import AudioSegment
import os

# Create the directory if it doesn't exist
output_directory = '/content/AUDIOBOOK/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Specify the folder for MP3 chunks
chunks_folder = 'INSERT PATH HERE'

# Function to combine audio chunks
def combine_chunks(chunk_paths, output_path):
    combined_audio = AudioSegment.empty()
    for i, chunk_path in enumerate(chunk_paths, start=1):
        try:
            print(f"Combining chunk {i} of {len(chunk_paths)}: {chunk_path}")
            chunk_audio = AudioSegment.from_mp3(chunk_path)
            combined_audio += chunk_audio
        except Exception as e:
            print(f"Error combining chunk {i}: {e}")
    combined_audio.export(output_path, format='mp3')
    print(f"Combined audio saved to: {output_path}")

# List all chunk files
chunk_files = [os.path.join(chunks_folder, file) for file in os.listdir(chunks_folder) if file.endswith('.mp3')]
chunk_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

# Split the list into parts of 100 chunks
chunk_parts = [chunk_files[i:i + 100] for i in range(0, len(chunk_files), 100)]

# Combine chunks in each part
for i, chunk_part in enumerate(chunk_parts, start=1):
    part_output_path = os.path.join(output_directory, f'part_{i}_audiobook.mp3')
    combine_chunks(chunk_part, part_output_path)


In [ ]:
from pydub import AudioSegment
import os

# Directory where the parts are stored
parts_directory = '/content/AUDIOBOOK/'

# List all part files
part_files = [os.path.join(parts_directory, file) for file in os.listdir(parts_directory) if file.startswith('part_') and file.endswith('.mp3')]
part_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

# Initialize an empty AudioSegment object
combined_audio = AudioSegment.empty()

# Combine all parts
for part_file in part_files:
    try:
        print(f"Adding {part_file} to the final audiobook")
        part_audio = AudioSegment.from_mp3(part_file)
        combined_audio += part_audio
    except Exception as e:
        print(f"Error processing {part_file}: {e}")

# Export the final combined audiobook
final_audiobook_path = '/content/AUDIOBOOK/full_audiobook.mp3'
combined_audio.export(final_audiobook_path, format='mp3')

print(f"Final audiobook created at: {final_audiobook_path}")


Adding /content/AUDIOBOOK/part_1_audiobook.mp3 to the final audiobook
Adding /content/AUDIOBOOK/part_2_audiobook.mp3 to the final audiobook


In [ ]:
import fitz  # PyMuPDF

# Define the PDF file path
pdf_path = 'INSERT PATH HERE'  # Replace with your PDF file path

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

# Call the function to extract text
pdf_text = extract_text_from_pdf(pdf_path)

# Calculate the number of characters
character_count = len(pdf_text)

# Print the result
print(f"Number of characters in the PDF: {character_count}")


# EXTRA

In [ ]:
import os
import re
import shutil
import fitz  # PyMuPDF
import time
from google.colab import drive
from elevenlabs import generate, set_api_key
from pydub import AudioSegment

# Key Variables for Easy Editing
pdf_path = 'INSERT PATH HERE'
chunks_colab_folder = '/content/AUDIOBOOK/Chunks/'
audiobook_colab_folder = '/content/AUDIOBOOK/Audiobook/'
chunks_drive_folder = '/content/drive/MyDrive/Audiobooks/Chunks/'
audiobook_drive_folder = '/content/drive/MyDrive/Audiobooks/Audiobook/'
api_key = "INSERT API KEY HERE"
voice_id = 'INSERT VOICE ID HERE'

# Connect to Google Drive if not connected
if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')

# Set your API key for Eleven Labs
set_api_key(api_key)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

# Function to upload chunks to Google Drive
def save_to_drive(file_path, drive_folder):
    try:
        shutil.copy(file_path, drive_folder)
        print(f"File uploaded to Google Drive: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Error uploading to Google Drive: {e}")

# Function to split text into chunks
def split_text_into_chunks(text, max_chunk_length=5000):
    chunks = []
    current_chunk = ""
    for sentence in re.split(r'(?<=[.!?])\s+', text):
        if len(current_chunk) + len(sentence) <= max_chunk_length:
            current_chunk += sentence
        else:
            chunks.append(current_chunk)
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

# Function to generate MP3 files
def generate_mp3(chunks, voice_id, chunks_folder, drive_folder):
    total_duration = 0
    total_characters_left = sum(len(chunk) for chunk in chunks)
    total_words_left = sum(len(re.findall(r'\w+', chunk)) for chunk in chunks)

    for i, chunk in enumerate(chunks, start=1):
        chunk_mp3_file_path = os.path.join(chunks_folder, f'{i}.mp3')
        if os.path.exists(chunk_mp3_file_path):
            print(f"MP3 for chunk {i} already exists, skipping...")
            continue

        try:
            audio = generate(text=chunk, voice=voice_id, model='eleven_multilingual_v1')
            with open(chunk_mp3_file_path, 'wb') as mp3_file:
                mp3_file.write(audio)
            chunk_duration = len(AudioSegment.from_file(chunk_mp3_file_path)) / 1000
            total_duration += chunk_duration
            total_characters_left -= len(chunk)
            total_words_left -= len(re.findall(r'\w+', chunk))
            save_to_drive(chunk_mp3_file_path, drive_folder)
            print(f"MP3 for chunk {i} generated successfully. Duration: {chunk_duration} seconds")
            print(f"Total Duration So Far: {total_duration} seconds")
            print(f"Characters Left: {total_characters_left} characters")
            print(f"Words Left: {total_words_left} words")
        except Exception as e:
            print(f"Exception occurred while generating MP3 for chunk {i}: {e}")
            break

    return total_duration

# Function to concatenate MP3 chunks
def concatenate_mp3_files(chunks_folder, audiobook_folder, file_name):
    audiobook = AudioSegment.silent(duration=0)  # Create an empty AudioSegment
    for mp3_file in sorted(os.listdir(chunks_folder), key=lambda x: int(x.split('.')[0])):
        audio_chunk = AudioSegment.from_mp3(os.path.join(chunks_folder, mp3_file))
        audiobook += audio_chunk

    audiobook_file_path = os.path.join(audiobook_folder, file_name)
    audiobook.export(audiobook_file_path, format="mp3")
    return audiobook_file_path

# Ensure necessary folders exist
os.makedirs(chunks_colab_folder, exist_ok=True)
os.makedirs(audiobook_colab_folder, exist_ok=True)
os.makedirs(chunks_drive_folder, exist_ok=True)
os.makedirs(audiobook_drive_folder, exist_ok=True)

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# Split the text into chunks
text_chunks = split_text_into_chunks(text)

# Generate MP3 files from text chunks
generate_mp3(text_chunks, voice_id, chunks_folder=chunks_colab_folder, drive_folder=chunks_drive_folder)

# Concatenate MP3 files into a single audiobook
audiobook_file_name = "Complete_Audiobook.mp3"
audiobook_path = concatenate_mp3_files(chunks_colab_folder, audiobook_colab_folder, audiobook_file_name)

# Upload the complete audiobook to Google Drive
save_to_drive(audiobook_path, audiobook_drive_folder)

print(f"Audiobook generation complete. Audiobook file saved as: {audiobook_file_name}")


In [ ]:
!rm -r /content/AUDIOBOOK

In [ ]:
from google.colab import files

# Specify the path to the MP3 file
mp3_path = '/content/AUDIOBOOK/part_1_audiobook.mp3'

# Create a download link for the MP3 file
files.download(mp3_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Specify the path to the MP3 file
mp3_path = '/content/AUDIOBOOK/part_2_audiobook.mp3'

# Create a download link for the MP3 file
files.download(mp3_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>